# Part I. SPARKIFY ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [37]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [38]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [39]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0],row[2],row[3],row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [40]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Creating NoSQL database after Extracting and Trasforming parts

## We are going to work with the <font color=red>event_datafile_new.csv</font> we created from the E and T parts located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Creating Cluster and Keyspace for Cassandra

#### Creating a Cluster

In [41]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [42]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [43]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### We now going finnaly to the backbone of the noSQL power! 

## We are going to create three tables based on the following three request (request that are going to be transformed into queries).



##### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


##### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

##### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## QUERY 1

In [44]:
#Creating table based on the query 1
query = "CREATE TABLE IF NOT EXISTS artist_library_by_session"
query = query + "(session_id int,item_in_session int, artist text,song text,length float, PRIMARY KEY (session_id,item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [50]:
#Insering the appropriate information to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO artist_library_by_session (session_id,item_in_session, artist,song,length)"
        query = query + "VALUES (%s, %s, %s, %s,%s)"
     
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))

In [52]:
#Running the query to check if we got what we wanted
query = "select artist,song,length from artist_library_by_session WHERE session_id = 338 and item_in_session= 4 "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## QUERY 2

In [53]:
#Creating table based on the query 2
query = "CREATE TABLE IF NOT EXISTS artist_library_by_user_id"
query = query + "(user_id int,session_id int,item_in_session int, artist text,song text,first_name text,last_name text, PRIMARY KEY ((user_id,session_id),item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)              

In [54]:
#Insering the appropriate information to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO artist_library_by_user_id (user_id,session_id,item_in_session, artist,song,first_name,last_name)"
        query = query + "VALUES (%s,%s, %s, %s, %s,%s,%s)"

        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

In [55]:
#Running the query to check if we got what we wanted
query = "select artist,song,first_name,last_name from artist_library_by_user_id WHERE user_id = 10 and session_id = 182 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## QUERY 3

In [56]:
#Creating table based on the query 3
query = "CREATE TABLE IF NOT EXISTS music_library_by_song"
query = query + "(song text,user_id int,session_id int,first_name text,last_name text, PRIMARY KEY ((song),user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)                

In [ ]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO music_library_by_song (song,user_id,session_id,first_name,last_name)"
        query = query + "VALUES (%s,%s, %s,%s,%s)"

        session.execute(query, (line[9],int(line[10]),int(line[8]),line[1],line[4]))

In [ ]:
# 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "select first_name,last_name from music_library_by_song WHERE song = 'All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

### SO GREAT! WE CREATED THE TABLES BASED ON THE QUERIES AND GOT THE RESULT WE WANTED!

### We drop the tables before closing out the sessions

In [35]:
query = "drop table artist_library_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table artist_library_by_user_id"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table music_library_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table artist_library_by_user_id"
Error from server: code=2200 [Invalid query] message="unconfigured table music_library_by_song"


### We close the session and cluster connection

In [36]:
session.shutdown()
cluster.shutdown()